Write A Function In Machine Code - Basically a String but more complicated

Linux memory == files?

memory mapped file - https://man7.org/linux/man-pages/man2/mmap.2.html

memory with code in must be marked "executable"

Code from https://github.com/tonysimpson/pointbreak >

In [1]:
import ctypes
import mmap


def create_callable_from_machine_code(machine_code, doc=None, restype=None, argtypes=None, use_errno=False, use_last_error=False):
    if argtypes is None:
        argtypes = []
    exec_code = mmap.mmap(-1, len(machine_code), prot=mmap.PROT_WRITE | mmap.PROT_READ | mmap.PROT_EXEC)
    exec_code.write(machine_code)
    c_type = ctypes.c_byte * len(machine_code)
    c_var = c_type.from_buffer(exec_code)
    address = ctypes.addressof(c_var)
    c_func_factory = ctypes.CFUNCTYPE(restype, *argtypes, use_errno=use_errno, use_last_error=use_last_error)
    func = c_func_factory(address)
    func._exec_code = exec_code # prevent GC of code
    func.__doc__ = doc
    return func


linux x86_64 calling convention

The calling convention of the System V AMD64 ABI is followed on GNU/Linux. The registers RDI, RSI, RDX, RCX, R8, and R9 are used for integer and memory address arguments and XMM0, XMM1, XMM2, XMM3, XMM4, XMM5, XMM6 and XMM7 are used for floating point arguments.

For system calls, R10 is used instead of RCX. Additional arguments are passed on the stack and the return value is stored in RAX.

https://www.felixcloutier.com/x86/

RET in hex C3

In [12]:
just_return = create_callable_from_machine_code(b'\xC3')

In [13]:
just_return()

In [14]:
import distorm3

In [20]:
return_21_code = b'\x48\xB8\x15\x00\x00\x00\x00\x00\x00\x00\xC3'
for offset, length, assembler, hex in distorm3.Decode(0, return_21_code, distorm3.Decode64Bits):
    print(f'{assembler}')

MOV RAX, 0x15
RET


In [22]:
return_21 = create_callable_from_machine_code(return_21_code, restype=ctypes.c_int64)

In [23]:
return_21()

21